Author: Paweł Chruszczewsju

Objective: The primary goal of this code is to develop a machine learning model that can classify the labels properly and address dataset imbalance. The dataset has three classes (1, 2, and 3). There are three problems in the challenge. In each problem, there is some specific task that needs to be done. In the following subsections, I describe three techniques I used to overcome the data imbalance problem.

Codes and libraries: This project requires Python  3. I have Used python 3.9. The following Python libraries are also required:

<li> numpy
<li> pandas
<li> matplotlib
<li> scikit-learn
<li> xgboost
<li> scipy
<li> seaborn
<li> itertools
<li> math
<li> mlxtend

In [239]:
import numpy as np
import pandas as pd
import warnings

## Plotting libraries
# import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Sklearn Libraries
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import RFECV
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.gaussian_process.kernels import DotProduct
# from sklearn.gaussian_process.kernels import Matern
# from sklearn.gaussian_process.kernels import RationalQuadratic
# from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc, \
            classification_report, recall_score, precision_recall_curve, roc_auc_score, precision_score, accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import get_scorer

## XGBoost Librarires
from xgboost import XGBClassifier

# pickle library
import pickle

## Scipy Libraries
from scipy.stats.mstats import winsorize
from scipy.stats import f
from scipy.stats import norm
from scipy.stats import chi2
from scipy.stats import ttest_ind
from scipy.stats import randint

#statistics
from statistics import stdev 

#itertools
from itertools import combinations, permutations

#mlxtend
from mlxtend.evaluate import paired_ttest_5x2cv
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

#math
import math

# Define random state
random_state = 2020
np.random.seed(random_state)
warnings.filterwarnings('ignore')

## Import Dataset & Initial Data Analysis

In [134]:
data = pd.read_csv('Dataset.csv')

In [155]:
data.drop(columns=['gdp_change'], inplace = True)

In [157]:
y.loc[y.isna().any()]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [136]:
data.head()

,Unnamed: 0,pd,y,atch,empch,salech,roech,ptbch,nich,dlcpdlttdebit,...,quickratio,bvdmv,nidseq,actdnat,ebitdxint,redsale,nidsale,ebitdsale,ltdat,ffodlt
0,1,2.585663e-20,0.0,0.275937,0.189045,0.132754,0.016827,1.190518,0.118336,2.508597,...,0.855730,0.405124,0.120632,1.475566,0.141434,0.130070,0.046393,0.091455,0.518975,0.233942
1,2,2.021916e-04,0.0,0.185610,0.016854,0.202329,-0.006262,-0.634720,0.143485,1.678391,...,0.862750,0.545357,0.114370,1.257911,0.121963,0.144769,0.051514,0.096485,0.428689,0.303986
2,3,1.698752e-03,0.0,0.212075,0.099448,0.165826,0.026716,0.143538,0.160463,2.079429,...,0.720055,0.505766,0.141086,1.384712,0.157226,0.168131,0.061078,0.100498,0.471844,0.261582
3,4,5.373830e-15,0.0,0.250723,0.276382,0.168910,0.003535,0.332061,0.077768,2.151250,...,0.778922,0.433039,0.144621,1.498123,0.175031,0.188591,0.061064,0.109792,0.518562,0.233980
4,5,5.429146e-23,0.0,0.090154,-0.043307,0.094780,-0.009272,0.745122,0.016664,2.265693,...,1.077016,0.327398,0.135348,1.453859,0.213208,0.210052,0.057668,0.105313,0.512129,0.221352


In [153]:
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,64285.0,44645.409069,28120.658134,7.710000e+02,1.966800e+04,4.309700e+04,68352.000000,96264.000000
pd,64285.0,0.059888,0.174016,8.938556e-188,1.474051e-24,1.646004e-09,0.002571,0.912808
atch,64285.0,0.150981,0.493803,-6.153846e-01,-4.053647e-02,5.274254e-02,0.184562,3.185871
empch,64285.0,0.085824,0.356288,-6.535496e-01,-4.444444e-02,2.155172e-02,0.126582,2.206522
salech,64285.0,0.491591,3.019083,-9.191919e-01,-3.418802e-02,7.044831e-02,0.215830,28.411765
roech,64285.0,-0.004649,2.057030,-1.073889e+01,-9.428077e-02,-4.720736e-03,0.057653,11.925383
ptbch,64285.0,-0.094815,9.646932,-5.594627e+01,-5.690631e-01,-2.517948e-03,0.502332,54.324612
nich,64285.0,0.013895,0.535532,-1.000000e+00,-2.379024e-01,4.158620e-02,0.268966,1.000000
dlcpdlttdebit,64285.0,2.417913,10.705792,-4.813566e+01,0.000000e+00,1.299619e+00,4.197935,59.401297
nwcdta,64285.0,0.153577,0.481716,-3.281553e+00,2.736148e-02,1.735873e-01,0.364889,0.852339


In [138]:
data.count().sort_values(ascending=False)

ffodlt           64285
ltdat            64285
pd               64285
y                64285
atch             64285
empch            64285
salech           64285
roech            64285
ptbch            64285
nich             64285
dlcpdlttdebit    64285
nwcdta           64285
redat            64285
ebitdat          64285
mvaluedtd        64285
saledat          64285
nidat            64285
dtdat            64285
lctdact          64285
quickratio       64285
bvdmv            64285
nidseq           64285
actdnat          64285
ebitdxint        64285
redsale          64285
nidsale          64285
ebitdsale        64285
Unnamed: 0       64285
dtype: int64

## Data Preprocessing

In [149]:
X = data.loc[:, data.columns != 'y']
y = data.loc[:, data.columns == 'y']

In [150]:
# changing extreme values(inf) to the 0.01 percentile and 0.99 percentile
def winsorize_all(predictors):
    for col in predictors.columns: 
         predictors[col] = winsorize(predictors[col], limits=0.01)
    return predictors

In [151]:
X = winsorize_all(X)

In [152]:
X.skew()

Unnamed: 0       0.176026
pd               3.454312
atch             3.674639
empch            3.161556
salech           8.503536
roech            0.735914
ptbch           -0.214624
nich            -0.114128
dlcpdlttdebit    0.748810
nwcdta          -4.449948
redat           -5.779699
ebitdat         -4.685178
mvaluedtd        4.953619
saledat          1.550603
nidat           -5.027924
dtdat            3.217487
lctdact          6.043565
quickratio       3.619294
bvdmv            1.082681
nidseq          -0.549455
actdnat          1.132375
ebitdxint        0.564292
redsale         -8.350867
nidsale         -8.124911
ebitdsale       -8.284005
ltdat            5.227732
ffodlt          -3.249561
dtype: float64

In [51]:
# # predictors distribution
# for i, col in enumerate(X.columns):
#     plt.figure(i)
#     sns.countplot(x=col, data=X);

In [52]:
# Perform first split
xtrain, xtest, ytrain, ytest = train_test_split(X, 
                                                y, 
                                                test_size=0.2, 
                                                stratify = y,
                                                random_state=42)

In [53]:
xtrain_pd = xtrain.loc[:,xtrain.columns == 'pd']
xtest_pd = xtest.loc[:,xtest.columns == 'pd']

In [54]:
xtrain_nopd = xtrain.loc[:,xtrain.columns != 'pd']
xtest_nopd = xtest.loc[:,xtest.columns != 'pd']

## Hyperparameter Tuning

### Naive Bayes

In [55]:
var_smoothing = np.logspace(-1,1, num=25)

In [56]:
gnb_grid = {'gaussiannb__var_smoothing': var_smoothing}

In [57]:
gnb = GaussianNB()

### Quadratic Discriminant Analysis

In [58]:
qda_params = [x for x in np.linspace(0.0, 2.5, 25)]

In [59]:
qda_grid = {'quadraticdiscriminantanalysis__reg_param': qda_params}

In [60]:
qda = QuadraticDiscriminantAnalysis()

### Logistic Regression

In [61]:
# the norm used in the penalization
logreg_penalty = ['l1', 'l2', 'elasticnet', None]

In [62]:
# Inverse of regularization strength
logreg_c = [0.1, 1, 10, 100, 1000]

In [63]:
# Algorithm to use in the optimization problem
logreg_solver = ['newton-cg','liblinear', 'saga', 'lbfgs']

In [64]:
logreg_weight = ['balanced', None]

In [65]:
logreg_grid = {'logisticregression__penalty' : logreg_penalty,
               'logisticregression__C' : logreg_c,
               'logisticregression__solver' : logreg_solver,
               'logisticregression__class_weight': logreg_weight}

In [66]:
logreg = LogisticRegression(random_state = random_state)

### Decision Trees

In [67]:
# Criterion to split on
dt_criterion = ['gini', 'entropy']

In [68]:
# The strategy used to choose the split at each node
# dt_splitter = ['best', 'random']

In [69]:
# Maximum number of levels in tree
dt_max_depth = [int(x) for x in np.linspace(1, 20, 5)]

In [70]:
# Add the default as a possible value
dt_max_depth.append(None)

In [71]:
# The minimum number of samples required to split an internal node
# dt_min_samples_split = [int(x) for x in np.linspace(2, 40, 10)]

In [72]:
# dt_min_impurity_decrease = [float(x) for x in np.linspace(0, 0.3, 6)]

In [73]:
# The minimum number of samples required to be at a leaf node
dt_min_samples_leaf = [int(x) for x in np.linspace(1, 5, 5)]

In [74]:
# Number of features to consider at every split
dt_max_features = ['auto', 'sqrt', 'log2']

In [75]:
# Weights associated with classes
dt_class = ['balanced_subsample', 'balanced', None]

In [76]:
dt_grid = {'decisiontreeclassifier__criterion': dt_criterion,
#            'decisiontreeclassifier__splitter': dt_splitter,
           'decisiontreeclassifier__max_depth': dt_max_depth,
#            'decisiontreeclassifier__min_samples_split': dt_min_samples_split,
#            'decisiontreeclassifier__min_impurity_decrease': dt_min_impurity_decrease,
           'decisiontreeclassifier__min_samples_leaf':dt_min_samples_leaf,
           'decisiontreeclassifier__max_features':dt_max_features,
           'decisiontreeclassifier__class_weight': dt_class}

In [77]:
dt = DecisionTreeClassifier(random_state = random_state)

### Random Forest

In [78]:
# Number of trees in Random Forest
rf_n_estimators = [int(x) for x in np.linspace(100, 300, 3)]
rf_n_estimators.append(10)
rf_n_estimators.append(50)
rf_n_estimators.append(1000)
rf_n_estimators.append(1500)

In [79]:
rf_max_depth = [int(x) for x in np.linspace(1, 20, 5)]

In [80]:
rf_max_depth.append(None)

In [81]:
rf_max_features = ['auto', 'sqrt', 'log2']

In [82]:
rf_min_samples_leaf = [int(x) for x in np.linspace(1, 5, 5)]

In [83]:
rf_criterion = ['gini', 'entropy']

In [84]:
# rf_min_samples_split = [int(x) for x in np.linspace(2, 40, 20)]

In [85]:
# rf_min_impurity_decrease = [float(x) for x in np.linspace(0, 0.3, 6)]

In [86]:
# Method of selecting samples for training each tree
# rf_bootstrap = [True, False]

In [87]:
rf_class = ['balanced_subsample', 'balanced', None]

In [88]:
rf_grid = {'randomforestclassifier__n_estimators': rf_n_estimators,
           'randomforestclassifier__max_depth': rf_max_depth,
           'randomforestclassifier__max_features': rf_max_features,
           'randomforestclassifier__criterion': rf_criterion,
#            'randomforestclassifier__min_samples_split': rf_min_samples_split,
#            'randomforestclassifier__min_impurity_decrease': rf_min_impurity_decrease,
           'randomforestclassifier__min_samples_leaf':rf_min_samples_leaf,
#            'randomforestclassifier__bootstrap': rf_bootstrap,
           'randomforestclassifier__class_weight': rf_class
          }

In [89]:
rf = RandomForestClassifier(random_state = random_state)

In [90]:
# base_models = [rdf]
# n_splits = 5
# grids = [rf_grid]
# lgb_stack = Create_classifier(n_splits = n_splits, base_models = base_models, grids = grids)        
# roc_auc_scores, feat_selected, feat_importance = lgb_stack.predict(xtrain, ytrain, xtest, ytest)

In [91]:
pip install psutil

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\pawel\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


### AdaBoost

In [92]:
# Maximum number of levels in tree
adab_max_depth = [1,2,5,10,20]

In [93]:
# Number of trees to be used
adab_n_estimators = [20,50,100,200,500]

In [94]:
# Learning rate
adab_eta = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1]

In [95]:
adab_algorithm = ['SAMME', 'SAMME.R']

In [96]:
adab_grid = {'adaboostclassifier__base_estimator__max_depth': adab_max_depth,
             'adaboostclassifier__n_estimators': adab_n_estimators,
             'adaboostclassifier__learning_rate': adab_eta,
             'adaboostclassifier__algorithm': adab_algorithm}

In [97]:
adab = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(random_state=random_state), random_state=random_state)

### XGBoost

In [98]:
# Number of trees to be used
xgb_n_estimators = [20,50,100,200,500]

In [99]:
# Maximum number of levels in tree
xgb_max_depth = [1,2,5,10,20]

In [100]:
# Minimum number of instaces needed in each node
xgb_min_child_weight = [1,2,3,4,5]

In [101]:
# Tree construction algorithm used in XGBoost
xgb_tree_method = ['auto', 'exact', 'approx']

In [102]:
# Learning rate
xgb_eta = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1]

In [103]:
# Minimum loss reduction required to make further partition
# xgb_gamma = [x for x in np.linspace(0, 0.5, 6)]

In [104]:
# Learning objective used
# xgb_objective = ['binary:logistic']

In [105]:
# xgb_lambda = [10,20,50,100]

In [106]:
# Balancing of positive and negative weights
xgb_weight = [119.85522788203754, None]

In [107]:
xgb_colsample_bytree = [x for x in np.linspace(0.1, 1, 5)]

In [108]:
subsample_bytree = [x for x in np.linspace(0.1, 1, 5)]

In [109]:
xgb_grid = {'xgbclassifier__n_estimators': xgb_n_estimators,
            'xgbclassifier__max_depth': xgb_max_depth,
            'xgbclassifier__min_child_weight': xgb_min_child_weight,
            'xgbclassifier__tree_method': xgb_tree_method,
            'xgbclassifier__learning_rate': xgb_eta,
#             'xgbclassifier__gamma': xgb_gamma,
#             'xgbclassifier__objective': xgb_objective,
#             'xgbclassifier__reg_lambda':xgb_lambda,
            'xgbclassifier__colsample_bytree':xgb_colsample_bytree,
            'xgbclassifier__subsample_bytree':subsample_bytree,
            'xgbclassifier__scale_pos_weight': xgb_weight}

In [110]:
xgb =  XGBClassifier(random_state = random_state, objective = 'binary:logistic')

### K-nearest Neighbors

In [111]:
knn_leaf_size = [int(x) for x in np.linspace(1, 55, 5)]

In [112]:
knn_weights = ['uniform','distance']

In [113]:
knn_n_neighbors = [int(x) for x in np.linspace(1, 30, 5)]

In [114]:
knn_p= [1, 2]

In [115]:
knn_metric = ['minkowski', 'euclidean', 'manhattan']

In [116]:
knn_grid = {'kneighborsclassifier__leaf_size':knn_leaf_size,
            'kneighborsclassifier__weights':knn_weights,
            'kneighborsclassifier__n_neighbors':knn_n_neighbors,
            'kneighborsclassifier__p': knn_p,
            'kneighborsclassifier__metric': knn_metric}

In [117]:
knn = KNeighborsClassifier()

### Multi-layer Perceptron

In [118]:
# for simplicity I choose 3 layers with the same number of neurons as there are features in my data set
mlp_hidden_layer_sizes = [(26,26)]

In [119]:
mlp_activation = ['tanh', 'relu', 'logistic']

In [120]:
mlp_solver = ['lbfgs', 'sgd', 'adam']

In [121]:
mlp_alpha = np.linspace(0.0001,0.1,10)

In [122]:
mlp_eta = ['constant','invscaling','adaptive']

In [123]:
mlp_grid = {'mlpclassifier__hidden_layer_sizes': mlp_hidden_layer_sizes,
            'mlpclassifier__activation': mlp_activation,
            'mlpclassifier__solver': mlp_solver,
            'mlpclassifier__alpha': mlp_alpha,
            'mlpclassifier__learning_rate': mlp_eta
}

In [124]:
mlp = MLPClassifier(random_state = random_state)

### SVC

In [125]:
# Inverse of regularization strength
svc_c = [0.1, 1, 10, 100, 1000]

In [126]:
# kernel selects the type of hyperplane used to separate the data; 
# ‘linear’ will use a linear hyperplane (a line in the case of 2D data). ‘rbf’ and ‘poly’ uses a non linear hyper-plane;
svc_kernel = ['linear', 'rbf', 'sigmoid']

In [127]:
# when kernel set to ‘poly’, the degree of the polynomial used to find the hyperplane to split the data
# svc_degree = [int(x) for x in np.linspace(0, 10, 10)]

In [128]:
# parameter for non linear hyperplanes
svc_gamma = [0.1, 1, 10, 100, 1000]
svc_gamma.append('scale')
svc_gamma.append(None)

In [129]:
svc_weight = ['balanced', None]

In [130]:
svc_grid = {'svc__C' : svc_c,
            'svc__kernel':svc_kernel,
#             'svc__degree':svc_degree,
            'svc__gamma':svc_gamma,
            'svc__class_weight':svc_weight}

In [131]:
svc = SVC(random_state = random_state, probability = True)

In [132]:
svc

SVC(probability=True, random_state=2020)

## Model Prediction

In [ ]:
chosen_set = ['ALL', 'PDE', 'PD']
base_models = [gnb, qda, logreg, dt, rf, adab, xgb, knn, svc, mlp]
n_splits = 4
grids = [gnb_grid, qda_grid, logreg_grid dt_grid, rf_grid, adab_grid, xgb_grid, knn_grid, svc_grid, mlp_grid]
lgb_stack = Create_classifier(n_splits = n_splits, base_models = base_models, grids = grids)        
roc_auc_scores, feat_selected, test_pred, estimators = lgb_stack.predict(xtrain, ytrain, xtest, ytest, chosen_set = chosen_set[0])

In [307]:
chosen_set = ['ALL', 'PDE', 'PD']
base_models = [gnb]
n_splits = 3
grids = [gnb_grid]
lgb_stack = Create_classifier(n_splits = n_splits, base_models = base_models, grids = grids)        
roc_auc_scores, feat_selected, test_pred, estimators = lgb_stack.predict(xtrain, ytrain, xtest, ytest, chosen_set = chosen_set[2])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   20.4s finished


IndexError: list index out of range

In [191]:
# predictions of all sets
predict_df = [test_pred, test_pred2, test_pred3]

In [195]:
lgb_stack = Create_classifier(n_splits = n_splits, base_models = base_models, grids = grids)        
predict_df_all = lgb_stack.joined_scores(predict_df)

In [306]:
class Create_classifier(object):
    def __init__(self, n_splits, base_models, grids):
        self.n_splits = n_splits
        self.base_models = base_models
        self.grids = grids

    def predict(self, x_train, y_train, x_test, y_test, chosen_set = ''):
        
        cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state = random_state)
                  
        roc_auc_scores = pd.DataFrame(columns = [str(i) for i in self.base_models])
        test_pred = pd.DataFrame(np.zeros((x_test.shape[0], len(self.base_models))), columns=[str(i).split('(')[0].lower() for i in self.base_models])
        test_pred.columns = pd.MultiIndex.from_product([[chosen_set], test_pred.columns])
        feat_selected = pd.DataFrame(np.zeros((len(x_train.columns), len(self.base_models))), index=x_train.columns, columns=[str(i) for i in self.base_models])
        feat_importance = pd.DataFrame(np.zeros((len(x_train.columns), len(self.base_models))), index=x_train.columns, columns=[str(i) for i in self.base_models])
        estimators = []         
        
        for i, clf in enumerate(self.base_models):
        
            pipe_lr = make_pipeline(PowerTransformer(method='yeo-johnson',standardize = True),
                                    clf)
            
            search = RandomizedSearchCV(estimator=pipe_lr, param_distributions = self.grids[i], cv = cv, n_jobs=-1, verbose=True, scoring = 'roc_auc', iid = True, refit = True, n_iter = 10)
            search = search.fit(x_train, y_train)
            
            filename = 'model_'+str(chosen_set)+str(clf)+'_saved.sav'
            pickle.dump(search.best_estimator_.steps[2][1], open(filename, 'wb'))
            
            estimators.append([chosen_set, i, search.best_estimator_.steps[2][1]])
            
            predict_rdf = search.best_estimator_.predict_proba(x_test)[:,1]
            test_pred[chosen_set][str(clf).split('(')[0].lower()] = predict_rdf.astype('float64')
                  
            roc_auc_scores.loc[0,str(clf)] = roc_auc_score(y_test, predict_rdf)
            
#             feat_est = search.best_estimator_.steps[1][1].k_feature_idx_
            
#             for j in feat_est:
#                 feat_selected.iloc[j,i] = 1
                  
#             for j in x_train.columns:
#                 feat_est = dict(zip(x_train.columns, search.best_estimator_.steps[1][1].k_feature_idx_))
#                 feat_selected.loc[str(j), str(clf)] = feat_est[str(j)]
                
#                 try:
#                     importances = dict(zip(x_train.columns, search.best_estimator_.named_steps[str(clf).split('(')[0].lower()].feature_importances_))
#                     feat_importance.loc[str(j), str(clf)] = importances[str(j)]
#                 except Exception:
#                     pass
                
        return roc_auc_scores, test_pred, estimators
    
    def joined_scores(self, predict_df):
    #         roc_auc_all = pd.concat(self.roc_auc)
            predict_df_all = pd.concat(predict_df, axis = 1)
            return predict_df_all

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# class Create_classifier(object):
#     def __init__(self, n_splits, base_models, grids):
#         self.n_splits = n_splits
#         self.base_models = base_models
#         self.grids = grids

#     def predict(self, x_train, y_train, x_test, y_test, chosen_set = ''):
        
#         cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state = random_state)
                  
#         roc_auc_scores = pd.DataFrame(columns = [str(i) for i in self.base_models])
#         test_pred = pd.DataFrame(np.zeros((x_test.shape[0], len(self.base_models))), columns=[str(i).split('(')[0].lower() for i in self.base_models])
#         test_pred.columns = pd.MultiIndex.from_product([[chosen_set], test_pred.columns])
#         feat_selected = pd.DataFrame(np.zeros((len(x_train.columns), len(self.base_models))), index=x_train.columns, columns=[str(i) for i in self.base_models])
#         feat_importance = pd.DataFrame(np.zeros((len(x_train.columns), len(self.base_models))), index=x_train.columns, columns=[str(i) for i in self.base_models])
#         models = []         
        
#         for i, clf in enumerate(self.base_models):
        
#             pipe_lr = make_pipeline(PowerTransformer(method='yeo-johnson',standardize = True),
#                                     RFECV(estimator = clf, step = 1, cv=cv, scoring = 'roc_auc'),
#                                     clf)
                  
#             search = RandomizedSearchCV(estimator=pipe_lr, param_distributions = self.grids[i], cv = cv, n_jobs=-1, verbose=True, scoring = 'roc_auc')
#             search.fit(x_train, y_train)
            
#             models.append([chosen_set, i, search.best_estimator_])
            
#             predict_rdf = search.best_estimator_.predict_proba(x_test)[:,1]
#             test_pred[chosen_set][str(clf).split('(')[0].lower()] = predict_rdf.astype('float64')
                  
#             roc_auc_scores.loc[0,str(clf)] = roc_auc_score(y_test, predict_rdf)
                  
#             for j in x_train.columns:
#                 feat_est = dict(zip(x_train.columns, search.best_estimator_.named_steps["rfecv"].ranking_))
#                 feat_selected.loc[str(j), str(clf)] = feat_est[str(j)]
                
#                 try:
#                     importances = dict(zip(x_train.columns, search.best_estimator_.named_steps[str(clf).split('(')[0].lower()].feature_importances_))
#                     feat_importance.loc[str(j), str(clf)] = importances[str(j)]
#                 except Exception:
#                     pass
                
#         return roc_auc_scores, feat_selected, feat_importance, test_pred, models

In [232]:
# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    p_val = np.log10(2) + norm.logsf(z, loc=0, scale=1) / np.log(10)
    
    p_val = math.exp(p_val)
    
    return p_val


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count, a = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

def calc_auc_ci(y_true, y_pred, alpha=0.95):
    auc, auc_cov = delong_roc_variance(y_true,y_pred)
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci_delong = norm.ppf(
        lower_upper_q,
        loc=auc,
        scale=auc_std)

    ci_delong[ci_delong > 1] = 1
    
    return ci_delong

In [351]:
import numpy as np
from scipy.stats import percentileofscore


def score_ci(
    y_true,
    y_pred,
    score_fun,
    n_bootstraps=2000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute confidence interval for given score function based on labels and predictions using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_pred: 1D list or array of predictions corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param confidence_level: Confidence level for computing confidence interval. (default: 0.95)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Score evaluated on labels and predictions, lower confidence interval, upper confidence interval, array of
    bootstrapped scores.
    """

    assert len(y_true) == len(y_pred)

    score = score_fun(y_true, y_pred)
    _, ci_lower, ci_upper, scores = score_stat_ci(
        y_true=y_true,
        y_preds=y_pred,
        score_fun=score_fun,
        n_bootstraps=n_bootstraps,
        confidence_level=confidence_level,
        seed=seed,
        reject_one_class_samples=reject_one_class_samples,
    )

    return score, ci_lower, ci_upper, scores


def score_stat_ci(
    y_true,
    y_preds,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=2000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute confidence interval for given statistic of a score function based on labels and predictions using
    bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_preds: A list of lists or 2D array of predictions corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param stat_fun: Statistic for which confidence interval is computed. (e.g. np.mean)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param confidence_level: Confidence level for computing confidence interval. (default: 0.95)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Mean score statistic evaluated on labels and predictions, lower confidence interval, upper confidence
    interval, array of bootstrapped scores.
    """

    y_true = np.array(y_true)
    y_preds = np.atleast_2d(y_preds)
    assert all(len(y_true) == len(y) for y in y_preds)

    np.random.seed(seed)
    scores = []
    for i in range(n_bootstraps):
        readers = np.random.randint(0, len(y_preds), len(y_preds))
        indices = np.random.randint(0, len(y_true), len(y_true))
        if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
            continue
        reader_scores = []
        for r in readers:
            reader_scores.append(score_fun(y_true[indices], y_preds[r][indices]))
        scores.append(stat_fun(reader_scores))

    mean_score = np.mean(scores)
    sorted_scores = np.array(sorted(scores))
    alpha = (1.0 - confidence_level) / 2.0
    ci_lower = sorted_scores[int(round(alpha * len(sorted_scores)))]
    ci_upper = sorted_scores[int(round((1.0 - alpha) * len(sorted_scores)))]
    
    ci_boot = np.array([ci_lower,ci_upper])
    
    return ci_boot


def pvalue(
    y_true,
    y_pred1,
    y_pred2,
    score_fun,
    n_bootstraps=2000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that score function for model I predictions is higher than for model II predictions
    using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_pred1: 1D list or array of predictions for model I corresponding to elements in y_true.
    :param y_pred2: 1D list or array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

    assert len(y_true) == len(y_pred1)
    assert len(y_true) == len(y_pred2)

    return pvalue_stat(
        y_true=y_true,
        y_preds1=y_pred1,
        y_preds2=y_pred2,
        score_fun=score_fun,
        n_bootstraps=n_bootstraps,
        two_tailed=two_tailed,
        seed=seed,
        reject_one_class_samples=reject_one_class_samples,
    )


def pvalue_stat(
    y_true,
    y_preds1,
    y_preds2,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=1000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that given statistic of score function for model I predictions is higher than for
    model II predictions using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_preds1: A list of lists or 2D array of predictions for model I corresponding to elements in y_true.
    :param y_preds2: A list of lists or 2D array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param stat_fun: Statistic for which p-value is computed. (e.g. np.mean)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

#     y_true = np.array(y_true)
#     y_preds1 = np.atleast_2d(y_preds1)
#     y_preds2 = np.atleast_2d(y_preds2)
#     assert all(len(y_true) == len(y) for y in y_preds1)
#     assert all(len(y_true) == len(y) for y in y_preds2)

    np.random.seed(seed)
#     t= []
    m = []
    sd = []
    score1 = []
    score2 = []
#     for i in range(n_bootstraps):
#         readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
#         readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
#         indices = np.random.randint(0, len(y_true), len(y_true))
#         if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
#             continue
#         reader_scores = []
#         for r in readers1:
#             reader_scores.append(score_fun(y_true[indices], y_preds1[r][indices]))
#         score1.append(reader_scores)
#         reader_scores2 = []
#         for r in readers2:
#             reader_scores.append(score_fun(y_true[indices], y_preds2[r][indices]))
#         score2.append(reader_scores2)

    for i in range(n_bootstraps):
        readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
        readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
        indices = np.random.randint(0, len(y_true), len(y_true))
        score1.append(roc_auc_score(y_true[indices], y_preds1[indices]))
        score2.append(roc_auc_score(y_true[indices], y_preds2[indices]))
                
    m.append(stat_fun(score1) - stat_fun(score2))
    
    s1 = (stdev(score1))
    n1 = len(score1)
    s2 = (stdev(score2))
    n2 = len(score2)
    
    sd.append(math.sqrt((s1**2)/n1 + (s2**2)/n2 - 2*np.correlate(score1,score2)*(s1/n1)*(s2/n2)))
#     sd.append(math.sqrt(s1 + s2)) 
    Z = m[0]/sd[0]
    
    p = norm.cdf(Z)
        
#     value, p = ttest_ind(score1, score2, equal_var=False)
    
#     if two_tailed:
#         p *= 2.0
    return p

def method(x,y):
    roc_auc_score(x,y)

def bootstrap_error_estimate(pred, truth, method, method_name="", alpha=0.95, sample_frac=0.5, iterations=2000):
    """
    Generate a bootstrapped estimate of confidence intervals
    :param pred: list of predicted values
    :param truth: list of experimental values
    :param method: method to evaluate performance, e.g. matthews_corrcoef
    :param method_name: name of the method for the progress bar
    :param alpha: confidence limit (e.g. 0.95 for 95% confidence interval)
    :param sample_frac: fraction to resample for bootstrap confidence interval
    :param iterations: number of iterations for resampling
    :return: lower and upper bounds for confidence intervals
    """
    index_list = range(0, len(pred))
    num_samples = int(len(index_list) * sample_frac)
    stats = []
    for _ in range(0, iterations):
        sample_idx = resample(index_list, n_samples=num_samples)
        pred_sample = [pred[x] for x in sample_idx]
        truth_sample = [truth[x] for x in sample_idx]
        stats.append(method(truth_sample, pred_sample))
    p = ((1.0 - alpha) / 2.0) * 100
    lower = max(0.0, np.percentile(stats, p))
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper = min(1.0, np.percentile(stats, p))
    
    ci_boot = np.array([lower,upper])
    
    return ci_boot

In [ ]:
# def pvalue_stat(
#     y_true,
#     y_preds1,
#     y_preds2,
#     score_fun,
#     stat_fun=np.mean,
#     n_bootstraps=2000,
#     two_tailed=True,
#     seed=None,
#     reject_one_class_samples=True,
# ):
#     """
#     Compute p-value for hypothesis that given statistic of score function for model I predictions is higher than for
#     model II predictions using bootstrapping.
#     :param y_true: 1D list or array of labels.
#     :param y_preds1: A list of lists or 2D array of predictions for model I corresponding to elements in y_true.
#     :param y_preds2: A list of lists or 2D array of predictions for model II corresponding to elements in y_true.
#     :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
#     :param stat_fun: Statistic for which p-value is computed. (e.g. np.mean)
#     :param n_bootstraps: The number of bootstraps. (default: 2000)
#     :param two_tailed: Whether to use two-tailed test. (default: True)
#     :param seed: Random seed for reproducibility. (default: None)
#     :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
#     need at least one positive and one negative sample. (default: True)
#     :return: Computed p-value, array of bootstrapped differences of scores.
#     """

#     y_true = np.array(y_true)
#     y_preds1 = np.atleast_2d(y_preds1)
#     y_preds2 = np.atleast_2d(y_preds2)
#     assert all(len(y_true) == len(y) for y in y_preds1)
#     assert all(len(y_true) == len(y) for y in y_preds2)

#     np.random.seed(seed)
#     z = []
#     for i in range(n_bootstraps):
#         readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
#         readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
#         indices = np.random.randint(0, len(y_true), len(y_true))
#         if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
#             continue
#         reader_scores = []
#         for r in readers1:
#             reader_scores.append(score_fun(y_true[indices], y_preds1[r][indices]))
#         score1 = stat_fun(reader_scores)
#         reader_scores = []
#         for r in readers2:
#             reader_scores.append(score_fun(y_true[indices], y_preds2[r][indices]))
#         score2 = stat_fun(reader_scores)
#         z.append(score1 - score2)

#     p = percentileofscore(z, 0.0, kind="weak") / 100.0
#     if two_tailed:
#         p *= 2.0
#     return p, z

In [249]:
predict_df_all[('ALL','gaussiannb')]

0        0.075678
1        0.001485
2        0.001420
3        0.001504
4        0.000188
           ...   
12875    0.001880
12876    0.000164
12877    0.000247
12878    0.000993
12879    0.003833
Name: (ALL, gaussiannb), Length: 12880, dtype: float64

In [347]:
y_preds1 = predict_df_all[('ALL','gaussiannb')]
y_preds2 = predict_df_all[('PDE','gaussiannb')]
y_true = ytest
y_true = np.array(y_true)
# y_preds1 = np.atleast_2d(y_preds1)
# y_preds2 = np.atleast_2d(y_preds2)
# assert all(len(y_true) == len(y) for y in y_preds1)
# assert all(len(y_true) == len(y) for y in y_preds2)
score1 = []
score2 = []

In [342]:
for i in range(2000):
    readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
    readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
    indices = np.random.randint(0, len(y_true), len(y_true))
    score1.append(roc_auc_score(y_true[indices], y_preds1[indices]))
    score2.append(roc_auc_score(y_true[indices], y_preds2[indices]))

In [265]:
del y_preds1, y_preds2, y_true, score1, score2

In [348]:
for i in range(1000):
    readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
    readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
    indices = np.random.randint(0, len(y_true), len(y_true))
    score1.append(roc_auc_score(y_true[indices], y_preds1[indices]))
    score2.append(roc_auc_score(y_true[indices], y_preds2[indices]))

In [343]:
value, p = ttest_ind(score1, score2, equal_var=False)

In [349]:
np.mean(score1)

0.9165044193720999

In [350]:
np.mean(score2)

0.9183600815147852

In [289]:
score1

[0.935227430252111,
 0.8834184797151268,
 0.9157848583512351,
 0.9280878479390618,
 0.9099754748373121,
 0.9103472408043967,
 0.8922910798122066,
 0.9220652096533067,
 0.925406429018976,
 0.9069771152807191]

In [280]:
a = []
for i in score1:
    a.append(i)

In [300]:
m = []

In [303]:
#         t.append(score1)
m.append(np.mean(score1) - np.mean(score2))
s1 = (stdev(score1)**2)/len(score1)
s2 = (stdev(score2)**2)/len(score2)

In [305]:
2*np.correlate(s1,s2)

ValueError: object of too small depth for desired array

In [ ]:
sd.append(sqrt(s1 + s2 - 2*np.correlate(s1,s2)*s1*s2))

In [207]:
def combined_ftest_5x2cv(estimator1, estimator2, x_train, y_train, scoring=None, random_seed=None):

    if isinstance(scoring, str):
        scorer = get_scorer(scoring)
    else:
        scorer = scoring

    variances = []
    differences = []

    def score_diff(X_1, X_2, y_1, y_2):

        estimator1.fit(X_1, y_1)
        estimator2.fit(X_1, y_1)
        est1_score = scorer(estimator1, X_2, y_2)
        est2_score = scorer(estimator2, X_2, y_2)
        score_diff = est1_score - est2_score
        return score_diff

    for i in range(5):

        X_1, X_2, y_1, y_2 = train_test_split(x_train, y_train, test_size=0.5, random_state=random_state)

        score_diff_1 = score_diff(X_1, X_2, y_1, y_2)
        score_diff_2 = score_diff(X_2, X_1, y_2, y_1)
        score_mean = (score_diff_1 + score_diff_2) / 2.
        score_var = ((score_diff_1 - score_mean)**2 + (score_diff_2 - score_mean)**2)

        differences.extend([score_diff_1**2, score_diff_2**2])
        variances.append(score_var)

    numerator = sum(differences)
    denominator = 2*(sum(variances))
    f_stat = numerator / denominator

    p_value = f.sf(f_stat, 10, 5)

    return float(f_stat), float(p_value)

In [208]:
estimators

[['ALL', 0, GaussianNB(var_smoothing=1.7782794100389228)]]

In [209]:
estimators[0][2]

GaussianNB(var_smoothing=1.7782794100389228)

In [222]:
class Scoring(object):
    def __init__(self, base_models):
#         self.roc_auc = roc_auc
        self.base_models = base_models

#     def joined_scores(self):
#         roc_auc_all = pd.concat(self.roc_auc)
#         predict_df_all = pd.concat(self.predict_df, axis = 1)
#         return predict_df_all

    def delong_test(self, predict_df_all, labels):
        
        Test_df_sets = pd.DataFrame(np.zeros((2, len(self.base_models))), index=['ALL/PDE','ALL/PD'], columns=[str(i).split('(')[0].lower() for i in self.base_models])
        Test_df_sets.columns = pd.MultiIndex.from_product([['DeLong Test'], Test_df_sets.columns])
        
        Test_df_all = pd.DataFrame(list(combinations(Test_df_sets['DeLong Test'].columns,2)),columns = ['1st Algorithm', '2nd Algorithm'])
        Test_df_all['score'] = 0
        Test_df_all.columns = pd.MultiIndex.from_product([['DeLong Test'], Test_df_all.columns])    
            
        for i, clf in enumerate(self.base_models):
        
            Test_df_sets['DeLong Test'].loc['ALL/PDE',str(clf).split('(')[0].lower()] = delong_roc_test(labels.values.ravel(), predict_df_all['ALL'][str(clf).split('(')[0].lower()], predict_df_all['PDE'][str(clf).split('(')[0].lower()])
            Test_df_sets['DeLong Test'].loc['ALL/PD',str(clf).split('(')[0].lower()] = delong_roc_test(labels.values.ravel(), predict_df_all['ALL'][str(clf).split('(')[0].lower()], predict_df_all['PD'][str(clf).split('(')[0].lower()])
        
        for j in range(Test_df_all.shape[0]):
            Test_df_all.loc[j, ('DeLong Test', 'score')]  = delong_roc_test(labels.values.ravel(), predict_df_all['ALL'][Test_df_all.loc[j, ('DeLong Test', '1st Algorithm')]], predict_df_all['ALL'][Test_df_all.loc[j, ('DeLong Test', '2nd Algorithm')]])
       
        return Test_df_sets, Test_df_all
    
    def bootstrap_test(self, predict_df_all, labels):
    
        Test_df_sets = pd.DataFrame(np.zeros((2, len(self.base_models))), index=['ALL/PDE','ALL/PD'], columns=[str(i).split('(')[0].lower() for i in self.base_models])
        Test_df_sets.columns = pd.MultiIndex.from_product([['Bootstrap Test'], Test_df_sets.columns])
        
        Test_df_all = pd.DataFrame(list(combinations(Test_df_sets['Bootstrap Test'].columns,2)),columns = ['1st Algorithm', '2nd Algorithm'])
        Test_df_all['score'] = 0
        Test_df_all.columns = pd.MultiIndex.from_product([['Bootstrap Test'], Test_df_all.columns])
            
        for i, clf in enumerate(self.base_models):
        
            Test_df_sets['Bootstrap Test'].loc['ALL/PDE',str(clf).split('(')[0].lower()] = pvalue(labels.values.ravel(), predict_df_all['ALL'][str(clf).split('(')[0].lower()], predict_df_all['PDE'][str(clf).split('(')[0].lower()], score_fun=roc_auc_score)
            Test_df_sets['Bootstrap Test'].loc['ALL/PD',str(clf).split('(')[0].lower()] = pvalue(labels.values.ravel(), predict_df_all['ALL'][str(clf).split('(')[0].lower()], predict_df_all['PD'][str(clf).split('(')[0].lower()], score_fun=roc_auc_score)
            
        for j in range(Test_df_all.shape[0]):
            Test_df_all.loc[j, ('Bootstrap Test', 'score')]  = pvalue(labels.values.ravel(), predict_df_all['ALL'][Test_df_all.loc[j, ('Bootstrap Test', '1st Algorithm')]], predict_df_all['ALL'][Test_df_all.loc[j, ('Bootstrap Test', '2nd Algorithm')]],score_fun=roc_auc_score)
       
        return Test_df_sets, Test_df_all
    
    def likelihood_RT(self, predict_df_all, labels, x_train):
    
#     Tu powinien być model, który zostanie wyszukany, a nie self.base_models

        Test_df_sets = pd.DataFrame((np.zeros((2, 1))), index=['ALL/PDE','ALL/PD'], columns=[str(self.base_models[0])])
        Test_df_sets.columns = pd.MultiIndex.from_product([['LRT'], Test_df_sets.columns])

        alt_log_likelihood = -log_loss(labels,
                                       predict_df_all['ALL'][str(self.base_models[0])],
                                       normalize=False)
        null_log_likelihood = -log_loss(ytest,
                                        predict_df_all['PDE'][str(self.base_models[0])],
                                        normalize=False)
        G = 2 * (alt_log_likelihood - null_log_likelihood)
        p_log_l = chi2.sf(G, x_train.shape[1])
        
        alt_log_likelihood = -log_loss(labels,
                                       predict_df_all['ALL'][str(self.base_models[0])],
                                       normalize=False)
        null_log_likelihood = -log_loss(ytest,
                                        predict_df_all['PD'][str(self.base_models[0])],
                                        normalize=False)
        
        G = 2 * (alt_log_likelihood - null_log_likelihood)
        p_log_2 = chi2.sf(G, x_train.shape[1])
        
        Test_df_sets['LRT'].loc['ALL/PDE' ,str(self.base_models[0])] = p_log_l
        Test_df_sets['LRT'].loc['ALL/PD', str(self.base_models[0])] = p_log_2

        return Test_df_sets
    
    def f_test(self, estimators_list, x_train, y_train, scoring, random_state):
    
        estimators = []
        for i in range(len(self.base_models)):
              estimators.append(estimators_list[i][2])
#             it should be 2 because of what models' list contains
#             estimators.append(estimators_list[i][2])
        estimators = list(combinations(estimators,2))
        
        p_values = []

        for i in range(len(estimators)): 
#           it should be eval becuase of the character of data imputed
#             estimator1 = eval(estimators[i][0])
#             estimator2 = eval(estimators[i][1])
            
            global estimator1, estimator2 
            estimator1 = estimators[i][0]
            estimator2 = estimators[i][1]
            
            f_stat, p_value = combined_ftest_5x2cv(estimator1, estimator2, x_train, y_train, scoring, random_state)
            
            p_values.append(p_value)
            
        f_p_values = pd.DataFrame(columns = ['algorithm1', 'algorithm2', 'score'])    
            
        for j in range(len(estimators)): 
            f_p_values.loc[j,'algorithm1'] = estimators[j][0]
            f_p_values.loc[j,'algorithm2'] = estimators[j][1]
            f_p_values.loc[j,'score'] = p_values[j]
        
        return(f_p_values)

In [211]:
def yeoj_graph(x_train, lbd_list, feature=''):
    
    plt.figure(figsize=(8,6))

    for i in range(len(lbd_list)):
        n_lines = len(lbd_list)
        c = np.arange(1, n_lines + 1)
        norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
        cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Blues)
        cmap.set_array([])
        a = x_train[feature].values.ravel()
        a = np.sort(a)
        b = stats.yeojohnson(x_train[feature], lmbda=lbd_list[i])
        b = np.sort(b)
        plt.plot(a,b, c=cmap.to_rgba(i + 1), label='λ = '+str(lbd_list[i]))
    plt.legend(loc=0)
    plt.ylabel("ψ(λ,x)", fontsize=15)
    plt.xlabel("x", fontsize=15)
    plt.savefig('yeo-johnson.png', dpi=1200)
    
    return plt.show()

In [200]:
def roc_comparison_all(predict_df, y_test):
    
    # Plot the figure
    # Train the models and record the results
    result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

    for i, (j, clf) in enumerate(predict_df):
        yproba = predict_df[j][clf]

        fpr, tpr, _ = roc_curve(y_test.values.ravel(),  yproba)
        auc = roc_auc_score(y_test.values.ravel(), yproba)

        result_table = result_table.append({'classifiers': [j,clf],
                                            'fpr':fpr, 
                                            'tpr':tpr, 
                                            'auc':auc}, ignore_index=True)

        result_table[['set', 'classifier']] = pd.DataFrame(result_table['classifiers'].tolist(), index=result_table.index)

    fig = plt.figure(figsize=(8,6))

    for k,m in enumerate(result_table.classifier.unique()):

    #     n_lines = len(base_models[:2])
    #     c = np.arange(1, n_lines + 1)
    #     norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
    #     cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Blues)
    #     cmap.set_array([])

        plt.plot(result_table.loc[result_table.set == 'ALL']['fpr'][k], 
                 result_table.loc[result_table.set == 'ALL']['tpr'][k],
    #              c=cmap.to_rgba(k + 2),
                 label="{}, AUC={:.3f}".format(str(m).split('(')[0].lower(), result_table.loc[result_table.set == 'ALL']['auc'][k]))

    plt.plot([0,1], [0,1], color='gray', linestyle='--')

    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("False Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)

    # plt.title('ROC Curve Logistic Regression Analysis', fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')
    plt.savefig('ALL.png',  dpi=1200)
    plt.close()

In [201]:
def roc_comparison_sets(predict_df, y_test):
    
    # Plot the figure
    # Train the models and record the results
    result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])
    for i, (j, clf) in enumerate(predict_df):
        yproba = predict_df[j][clf]

        fpr, tpr, _ = roc_curve(y_test.values.ravel(),  yproba)
        auc = roc_auc_score(y_test.values.ravel(), yproba)

        result_table = result_table.append({'classifiers': [j,clf],
                                            'fpr':fpr, 
                                            'tpr':tpr, 
                                            'auc':auc}, ignore_index=True)

        result_table[['set', 'classifier']] = pd.DataFrame(result_table['classifiers'].tolist(), index=result_table.index)

        fig = plt.figure(figsize=(8,6))

        for k,m in enumerate(result_table.set.unique()):

            plt.plot(result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['fpr'].values[0], 
                     result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['tpr'].values[0],
                     label="{}, AUC={:.3f}".format(str(m), result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['auc'].values[0]))

            plt.plot([0,1], [0,1], color='gray', linestyle='--')

            plt.xticks(np.arange(0.0, 1.1, step=0.1))
            plt.xlabel("False Positive Rate", fontsize=15)

            plt.yticks(np.arange(0.0, 1.1, step=0.1))
            plt.ylabel("True Positive Rate", fontsize=15)

            plt.legend(prop={'size':13}, loc='lower right')
            
        plt.savefig(str(clf)+str(m)+'.png',  dpi=1200)
        plt.close()

In [216]:
def graph_ci_alternative(predict_df, base_models):
    
        
    result_table = pd.DataFrame(columns=['classifiers', 'delong','bootstrap'])
    for i, (j, clf) in enumerate(predict_df):
        yproba = predict_df[j][clf]

        delong = calc_auc_ci(ytest.values.ravel(),  yproba, alpha=0.95) 
        bootstrap = score_stat_ci(ytest.values.ravel(), yproba,  roc_auc_score)

        result_table = result_table.append({'classifiers': [j,clf],
                                            'delong':delong, 
                                            'bootstrap':bootstrap}, ignore_index=True)

        result_table[['set', 'classifier']] = pd.DataFrame(result_table['classifiers'].tolist(), index=result_table.index)

    for k,m in enumerate(result_table.set.unique()):
    
        plt.figure(figsize=(8,6))

        SMALL_SIZE = 10
        MEDIUM_SIZE = 12
        BIGGER_SIZE = 14

        plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        x_ticks = [str(i).split('(')[0].lower() for i in base_models]

        for n,l in enumerate(result_table.classifier.unique()):

            eb_1 = plt.errorbar(x=n+1, 
                             y=(result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['bootstrap'].values[0][1] + result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['bootstrap'].values[0][0])/2, 
                             yerr=[(result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['bootstrap'].values[0][1] - result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['bootstrap'].values[0][0])/2],
                             fmt='ok',
                             capsize = 10)

            eb_2 = plt.errorbar(x=n+1.1, 
                             y=(result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['delong'].values[0][1] + result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['delong'].values[0][0])/2, 
                             yerr=[(result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['delong'].values[0][1] - result_table.loc[(result_table.classifier == str(l)) & (result_table.set == str(m))]['delong'].values[0][0])/2],
                             fmt='ok',
                             capsize = 10)
            eb_2[-1][0].set_linestyle('--')
            
            # I need to manipulate 3rd parameter in arange, so the graph looks nice & I also need to do the same in errorbar(x)

            plt.xticks(np.arange(1.05,len(x_ticks)+0.5,1), x_ticks, rotation=90)
            plt.tight_layout()

            plt.ylabel("ROC AUC Przedział Ufności", fontsize=15)
            plt.tight_layout()

        plt.savefig('plot'+str(m)+'ci.png', dpi=1200)
        plt.close()

In [203]:
def graph_ci(predict_df, base_models):
    
        
    result_table = pd.DataFrame(columns=['classifiers', 'delong','bootstrap'])
    for i, (j, clf) in enumerate(predict_df):
        yproba = predict_df[j][clf]

        delong = calc_auc_ci(ytest.values.ravel(),  yproba, alpha=0.95) 
        bootstrap = bootstrap_error_estimate(yproba, ytest.values.ravel(), roc_auc_score)

        result_table = result_table.append({'classifiers': [j,clf],
                                            'delong':delong, 
                                            'bootstrap':bootstrap}, ignore_index=True)

        result_table[['set', 'classifier']] = pd.DataFrame(result_table['classifiers'].tolist(), index=result_table.index)
        
        plt.figure(figsize=(8,6))

        for k,(m,n) in enumerate(result_table.classifiers):

            SMALL_SIZE = 10
            MEDIUM_SIZE = 12
            BIGGER_SIZE = 14

            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            x_ticks = ('Wszystkie Zmienne', 'Bez PD', 'Tylko PD')
            
            def m_value(x):
                if x == 'ALL':
                    x_1 = 1
                    x_2 = x_1 + 0.1
                elif x == 'NOPD':
                    x_1 = 2
                    x_2 = x_1 + 0.1
                elif x == 'PD':
                    x_1 = 3
                    x_2 = x_1 + 0.1
                return list([x_1,x_2])

            eb_1 = plt.errorbar(x=m_value(str(m))[0], 
                             y=(result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['bootstrap'].values[0][1] + result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['bootstrap'].values[0][0])/2, 
                             yerr=[(result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['bootstrap'].values[0][1] - result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['bootstrap'].values[0][0])/2],
                             fmt='ok',
                             capsize = 10)

            eb_2 = plt.errorbar(x=m_value(str(m))[1], 
                             y=(result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['delong'].values[0][1] + result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['delong'].values[0][0])/2, 
                             yerr=[(result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['delong'].values[0][1] - result_table.loc[(result_table.classifier == str(clf)) & (result_table.set == str(m))]['delong'].values[0][0])/2],
                             fmt='ok',
                             capsize = 10)
            eb_2[-1][0].set_linestyle('--')

            plt.xticks([1.05,2.05,3.05], x_ticks, rotation=90)
            plt.tight_layout()


            plt.ylabel("ROC AUC Przedział Ufności", fontsize=15)
            plt.tight_layout()

        plt.savefig('plot'+str(clf)+'ci.png', dpi=1200)
        plt.close()

### How to refer to 5x2 CV test below

In [335]:
lgb_score = Scoring(base_models = base_models)

In [354]:
a,b = lgb_score.delong_test(predict_df_all, ytest)

In [355]:
c,d = lgb_score.bootstrap_test(predict_df_all, ytest)

In [360]:
c

,Bootstrap Test
,gaussiannb
ALL/PDE,3.177230e-45
ALL/PD,1.000000e+00


In [331]:
roc_auc_scores3

,GaussianNB()
0,0.904118


In [358]:
c

,Bootstrap Test
,gaussiannb
ALL/PDE,3.177230e-45
ALL/PD,1.000000e+00


In [359]:
d

Empty DataFrame
Columns: [(Bootstrap Test, 1st Algorithm), (Bootstrap Test, 2nd Algorithm), (Bootstrap Test, score)]
Index: []

In [ ]:
f_p_values = lgb_score.f_test(models, xtrain, ytrain, scoring = 'roc_auc', random_state = random_state)

In [ ]:
graph_ci_alternative(test_pred, base_models)

In [225]:
a = graph_ci_alternative(predict_df_all, ytest)

In [291]:
b = graph_ci_alternative(predict_df_all, ytest)

In [ ]:
# lgb_stack = Create_classifier(n_splits = n_splits, base_models = base_models, grids = grids)        
# roc_auc_scores, feat_selected, test_pred, models = lgb_stack.predict(xtrain, ytrain, xtest, ytest, chosen_set = chosen_set[0])